In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
#pip install openpyxl
pd.set_option('display.max_columns', None)

# Pendientes:
- Antonio: preguntarle si las posibilidades que voy a tener en tipo de servicio va a ser MPLS vs internet (DIA y B4B), para ajustar la función `def Commercial_model_changes()`. ahora he puesto "Other services quoted".

- normalización de proveedores conocidos (cargar_y_procesar_excel_ds)

- De momento, al read_csv, eliminamos las "alternativas" para poder analizar la desviación de precios y nos quedamos sólo con las "response to request"--> 'Does this offer match the customer request'

- Hacer el script de `src` y `.py`

- `Next steps`: 
    - tipo de cambio: para que los precios sean comparables, se pone el tipo de cambio del día de ejecución del script. Pero tengo dos fechas posibles de las que tirar: 
        - Date of Request
        - Response expected Date by Requester 
    - revisar el diccionario de las divisas, a ver si él cree que habría que meter alguna más


# Vistos:

- preguntar a Antonio = 
    - Si cree que puedo completar la divisa principal (main access) con la del back up o con la de la maintenance : ❌``NO`` esto no se puede. --> CAMBIARLO en el `script`
    - entiendo que el PE siempre va a tener su divisa completa en main accesss y que no voy a tener que completar nada --> ✅ correcto


In [ ]:
def cargar_y_procesar_excel_bend(ruta_archivo = "bend.xlsm", hoja = "B-Ends"):
    """Carga y procesa el archivo Excel B-End.

    Realiza limpieza de columnas, formatea strings y filtra las ofertas válidas.

    Args:
        ruta_archivo (str): Ruta al archivo Excel. Por defecto: "bend.xlsm"
        hoja (str): Nombre de la hoja del Excel a cargar. Por defecto: "B-Ends"

    Returns:
        pd.DataFrame: DataFrame procesado y listo para análisis.
    """

    #skiprows porque empieza en la fila 7
    bend = pd.read_excel(ruta_archivo, sheet_name=hoja, skiprows = 6, header=1) 

    #Limpieza de columnas:
    bend.columns = bend.columns.str.strip() 
    bend.rename(columns = {'Unnamed: 0': 'quotation_ID'}, inplace = True)
    
    #Formateo de columnas:
    bend['Main Access Currency'] = bend['Main Access Currency'].str.lower().str.strip()
    bend['City'] = bend['City'].str.title()
    bend['Country'] = bend['Country'].str.replace('_', ' ')

    #Columna nueva para identificar el origen
    bend['Source'] = 'Request B-End'

    # Filtrar: solo 'Response to Request' para poder analizar la desviación de precios. Confirmar con Antonio.
    bend = bend[bend['Does this offer match the customer request'] == 'Response to Request'] 

    # Ordenar por 'Option ID' y resetear índice
    bend.sort_values(by='Option ID', ascending=True, inplace=True) 

    #Cremos una nueva columna de índice 'id' y renombramos columnas
    bend = bend.reset_index(drop=True) #eliminamos el índice que salía por defecto de pandas para crear uno nuevo que va a ser la igual en los tres dataframes, por si el unique ID no coincide
    bend = bend.reset_index().rename(columns = {'index': 'id', 'Comments VPN Site Info\n/Commercial Model': 'Commercial Model'})
    
    return bend

In [3]:
bend = cargar_y_procesar_excel_bend()

C:\Users\alcorce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
def commercial_model_provided(celda):
    """Función para establecer el servicio prestado en una sola columna. El servicio prestado sobreescribe a la columna 'Comments VPN Site Info\n/Commercial Model'.

    Args:
        celda (string): cada valor de la columna 'Comments VPN Site Info\n/Commercial Model'

    Returns:
        string: nuevo valor que va a tomar la celda que podrá ser B4B, DIA o MPLS en función del servicio que se preste. 
    """
    if celda == 'B4B Resale-Unmanaged':
        return "B4B"
    elif celda == 'DIA Resale-Unmanaged':
        return "DIA"
    else: 
        return "MPLS" #preguntar a Antonio si aquí podemos tener también el servicio de Ethernet.

In [5]:
#Llamamos a la función "service_provided" y cambiamos el nombre de la columna:
bend['Commercial Model'] = bend['Commercial Model'].apply(commercial_model_provided)

In [6]:
bend_summary = bend[['id', 'quotation_ID', 'Source',
                    'Standard Services', 'Site ID', 'Lot', 'City', 'Country', 
                    'Main access speed UpStream (Kbps)', 'Main access speed DownStream (kbps)', 
                    'Main access technology', 'Main Access Guaranteed Bandwidth %', 
                    'Commercial Model','Contract Term (month)', 
                    'Main Access Provider (last mile Provider)', 'Main Access Currency']]

In [7]:
#bend_summary.to_csv('bend_summary.csv', index=False)

In [8]:
def completing_currency(df, columna_a_completar, columna_backup_1, columna_backup_2):
    """
    Completa valores faltantes (NaN) en una columna "objetivo" usando dos columnas de respaldo, en orden de prioridad. La prioridad es:
        1. Usar la columna principal si tiene valor.
        2. Si está vacía, usar el valor de columna_backup_1.
        3. Si también está vacía el backup_1, usar columna_backup_2.

    Args:
        df (pd.DataFrame): El DataFrame sobre el cual trabajar.
        columna_a_completar (str): Nombre de la columna principal que queremos completar.
        columna_backup_1 (str): Primera columna de respaldo (prioridad 1).
        columna_backup_2 (str): Segunda columna de respaldo (prioridad 2, si la prioridad 1 está vacía).

    Returns:
        pd.Series: Una serie con los valores completados.
    """
    id_dato_faltante = [] 

    def completar(row):
        
        if pd.isna(row[columna_a_completar]):

            id_dato_faltante.append(row['quotation_ID'])

            if pd.notna(row[columna_backup_1]):
                return row[columna_backup_1]
            
            elif pd.notna(row[columna_backup_2]):
                return row[columna_backup_2]
            
        return row[columna_a_completar]
    
    #Aplicamos la función:
    result =  df.apply(completar, axis=1)

    # Imprimo los ID conflictivos:
    print(f"🔍❌Ids con dato faltante: '{len(id_dato_faltante)}' \n {id_dato_faltante}")

    return result

In [9]:
def cargar_y_procesar_ds(ruta_archivo="ds.xlsx"):
    """Carga y procesa el archivo Deal Specialist.

    Args:
        ruta_archivo (str): Ruta al archivo Excel. Por defecto: "ds.xlsm"

    Returns:
        pd.DataFrame: DataFrame procesado y listo para análisis.
    """

    ds = pd.read_excel(ruta_archivo, index_col=0)

    #Limpieza de columnas:
    ds.columns = ds.columns.str.strip() 
    ds.rename(columns = {'Unnamed: 1': 'quotation_ID'}, inplace = True)

    # Completar 'Main Access Currency' (OJO🔍 REVISAR CON ANTONIO CUANDO YA TENGAMOS DATOS COMPLETOS)
    #quitar cuando sepamos de dónde viene el problema
    ds['Main Access Currency'] = completing_currency(
        df = ds,
        columna_a_completar = 'Main Access Currency',
        columna_backup_1 = 'Back Up Maintenance Currency',
        columna_backup_2 = 'Main Maintenance Currency')
    
    #Formateo de columnas:
    ds['Main Access Currency'] = ds['Main Access Currency'].str.lower().str.strip()
    ds['City'] = ds['City'].str.title()
    ds['Country'] = ds['Country'].str.title().str.replace('_', ' ').str.replace('Brasil', 'Brazil')

    #Columna nueva para identificar el origen
    ds['Source'] = 'Deal Specialist'

    # Normalizamos los proveedores conocidos:
    ds['Main Access Provider (last mile Provider)']= ds['Main Access Provider (last mile Provider)'].str.replace('Telefonica_Chile_S.A._(Mayorista)', 'Telefonica Chile S.A.').str.replace('INTELIGLOBE', 'Inteliglobe').str.replace("_", " ")

    #Creamos la columna Full contract Value:
    ds['FCV'] = ds['Main Access NRC'] + (ds['Contract Term (month)']* ds['Main Access MRC'])

    # Extraemos commercial Model
    ds['Commercial_Model']= ds['Comments VPN Site Info\n/Commercial Model'].str.split(' ', expand = True).get(0)

    # Ordenar por 'Option ID' y resetear índice
    ds.sort_values(by='Option ID', ascending=True, inplace=True)  
    
    #Cremos una nueva columna de índice 'id' y renombramos columnas
    ds = ds.reset_index(drop=True) #eliminamos el índice inicial de pandas
    ds = ds.reset_index().rename(columns = {'index': 'id'})

    return ds

In [10]:
ds = cargar_y_procesar_ds()

🔍❌Ids con dato faltante: '67' 
 ['2611E0725E2B25F556A814F73B59873D', '427209309476D3A41897F85193CF0FBD', '5E0052597800BB49ED3B1C81B205B70B', 'A4D3E236BAB37ADD47BF98D910D6636D', '304DD4FA60EAD472BF68B8A400F025A2', '73FD7797AA3AB004AA5B333037B601FA', 'F544F1AC0A69E608002497E8A1187D58', '21B7980C34CFE90076B5D6274AAE81D1', '140DDE2C67440E37BF4C9B6B9DFD6E3E', 'F8345872CEB5E92BCCA9A131B65C0999', '67A4C057CED40E5E1029D77F4F3A556C', '3CF268267A056338E9432F72137A7D56', '7CD82B34C0D17DF3DE38A2003890A5EF', '675C5F8F73CE92C09EEE0FD67150F2D6', 'A440C1919780F657E8B6728C86F36432', '3C03BD7DDCB1628D44641B904158442F', '6DD12EF1A3E782743363C7C4E9780C5D', '1EEDF79D42937655684A95BDE752B869', '221BE4C5CFFA00A32EFA0E7D7957C5F7', '31FDA691B1D36A47BEED6B5B27DF0503', 'D158BFC732E468ACFB291DF7E5797CF2', '135321A3E409780E74AA57367C033E0E', 'B6611C71B03DFA600AC77B77BCD30EE7', 'FA7D3EC243BBDC10C224017525A2077C', 'FA353A27F9E5A8A219FBCB2110876504', 'AE979FD1787181595F2CF589399BA16F', '6FD95FA8DE042226CE242BCFCB3BC1

In [11]:
ds_summary = ds[['id', 'quotation_ID', 'Source', 
                'Standard Services', 'Site ID', 'Lot', 'City', 'Country', 
                'Main access speed UpStream (Kbps)', 'Main access speed DownStream (kbps)', 
                'Main access technology', 'Main Access Guaranteed Bandwidth %', 
                'Commercial_Model','Contract Term (month)', 
                'Main Access Provider (last mile Provider)', 'Main Access Currency',
                'Main Access NRC', 'Main Access MRC', 'FCV']]

In [12]:
#ds_summary.to_csv('ds_summary.csv', index=False)

In [13]:
def cargar_y_procesar_pe (ruta_archivo = "pe.csv"):
    """Carga y procesa el archivo csv generado por el Pricing Engine

    Args:
        ruta_archivo (str): Ruta al archivo CSV. Por defecto: "pe.csv"

    Returns:
        pd.DataFrame: DataFrame procesado y listo para análisis.
    """

    pe = pd.read_csv(ruta_archivo, sep=';')

    #Limpieza de columnas:
    pe.columns = pe.columns.str.strip() 

    #Generamos el ID único a partir de otras celdas:
    pe['quotation_ID'] = pe['unique_id'].str.split(' ', expand = True).get(2).str.split('_', expand = True).get(0)
    
    
    #Formateo de columnas:
    pe['main_access_currency_cd'] = pe['main_access_currency_cd'].str.lower().str.strip()
    pe['country_name'] = pe['country_name'].str.replace('_', ' ')
    pe['Source'] = 'Pricing Engine' #creamos una columna
    
    # Normalizamos los proveedores conocidos:
    pe['main_access_provider_name'] = pe['main_access_provider_name'].str.replace('Telefonica_Empresas_Chile_S.A.','Telefonica Chile S.A.').str.replace('Inteliglobe_Communications_USA', 'Inteliglobe').str.replace("_", " ")

    #Creamos la columna Full contract Value
    pe['FCV'] = pe['main_access_nrc_amt'] + (pe['contract_term_cd']* pe['main_access_mrc_amt'])

    # Extraemos commercial Model
    pe['Commercial_Model'] = pe['vpn_site_comments_des'].str.split(' ', expand = True).get(0)

    #Ordenar por 'Option ID' y resetear índice
    pe.sort_values(by='option_id', ascending=True, inplace=True) 

    #Cremos una nueva columna de índice 'id' y renombramos columnas
    pe = pe.reset_index(drop=True) 
    pe = pe.reset_index().rename(columns = {'index': 'id'})

    return pe

In [14]:
pe = cargar_y_procesar_pe()

In [15]:
pe_summary = pe[['id', 'quotation_ID', 'Source', 
                'standard_services_cd', 'site_id', 'lot_cd', 'city_name', 'country_name',
                'main_access_speed_upstream_kbps_qt', 'main_access_speed_downstream_kbps_qt', 
                'main_access_technology_name', 'main_access_guaranteed_bandwidth_qt', 
                'Commercial_Model', 'contract_term_cd',
                'main_access_provider_name','main_access_currency_cd',
                'main_access_nrc_amt_quoted_by', 'main_access_nrc_amt',
                'main_access_mrc_amt_quoted_by', 'main_access_mrc_amt', 'FCV']]

In [16]:
pe_summary =  pe_summary.rename(columns = {
                                'Source': 'Source',
                                'standard_services_cd': 'Standard Services',
                                'site_id': 'Site ID',
                                'lot_cd': 'Lot',
                                'city_name': 'City',
                                'country_name': 'Country',
                                'main_access_speed_upstream_kbps_qt': 'Main access speed UpStream (Kbps)',
                                'main_access_speed_downstream_kbps_qt': 'Main access speed DownStream (kbps)',
                                'main_access_technology_name': 'Main access technology',
                                'main_access_guaranteed_bandwidth_qt': 'Main Access Guaranteed Bandwidth %',
                                'contract_term_cd': 'Contract Term (month)',
                                'main_access_nrc_amt':'Main Access NRC',
                                'main_access_mrc_amt': 'Main Access MRC',
                                'main_access_provider_name': 'Main Access Provider (last mile Provider)',
                                'main_access_currency_cd': 'Main Access Currency'
                                })

In [17]:
#pe_summary.to_csv('pe_summary.csv', index=False)

# Concatenación:

- bend_summary
- ds_summary
- pe_summary

In [18]:
print(bend_summary.shape)
print(ds_summary.shape)
print(pe_summary.shape)

(275, 16)
(275, 19)
(275, 21)


In [19]:
#Axis = 0: apila un df debajo del anterior, porque concatena por filas. Para concatenar por columnas (poner el df a la dcha, usamos merge-infra-)
df_concat = pd.concat([bend_summary, ds_summary, pe_summary], axis = 0, ignore_index = True)

In [20]:
df_concat.shape

(825, 22)

In [21]:
df_concat.tail(1)

,id,quotation_ID,Source,Standard Services,Site ID,Lot,City,Country,Main access speed UpStream (Kbps),Main access speed DownStream (kbps),Main access technology,Main Access Guaranteed Bandwidth %,Commercial Model,Contract Term (month),Main Access Provider (last mile Provider),Main Access Currency,Commercial_Model,Main Access NRC,Main Access MRC,FCV,main_access_nrc_amt_quoted_by,main_access_mrc_amt_quoted_by
824,274,23834179B18B5D282CA33238DDB75BF2,Pricing Engine,Internet,VEN-06,Internet Ppal,Barcelona,Venezuela,50000.0,50000.0,Ethernet Access,1.0,NaN,36,Telefonica Venezuela (TELCEL),dollar,DIA,3995.0,1293.50385,50561.1386,costbook,costbook


In [22]:
df_concat.groupby('Source')['quotation_ID'].count()

Source
Deal Specialist    275
Pricing Engine     275
Request B-End      275
Name: quotation_ID, dtype: int64

In [23]:
df_concat[(df_concat['Source'] != 'Request B-End') & (df_concat['Standard Services'] != 'Internet')].sample(2)

,id,quotation_ID,Source,Standard Services,Site ID,Lot,City,Country,Main access speed UpStream (Kbps),Main access speed DownStream (kbps),Main access technology,Main Access Guaranteed Bandwidth %,Commercial Model,Contract Term (month),Main Access Provider (last mile Provider),Main Access Currency,Commercial_Model,Main Access NRC,Main Access MRC,FCV,main_access_nrc_amt_quoted_by,main_access_mrc_amt_quoted_by
281,6,2611E0725E2B25F556A814F73B59873D,Deal Specialist,Unmanaged VPN MPLS,BRA-01,MPLS,Raposo,Brazil,200000.0,200000.0,Ethernet Access,NaN,NaN,36,Telefonica Brasil (VIVO Empresas/GVT),dollar,NaN,599.0,180.000,7079.0,NaN,NaN
556,6,2611E0725E2B25F556A814F73B59873D,Pricing Engine,Unmanaged VPN MPLS,BRA-01,MPLS,Sao Paulo,Brazil,200000.0,200000.0,Ethernet Access,NaN,NaN,36,BRFIBRA (Compuline) Brazil,dollar,NaN,250.0,207.575,7722.7,apis,apis


In [24]:
df_concat[df_concat['Source'] != 'Request B-End'][['Standard Services', 'FCV']].sample(10)

,Standard Services,FCV
681,Internet,6440.918512
536,Internet,2750.000000
359,Internet,6192.000000
311,Internet,8731.200000
277,Internet,11200.000000
402,Internet,2606.000000
478,Internet,5450.000000
380,Internet,2606.000000
635,Internet,10080.000000
532,Internet,2750.000000


In [25]:
df_concat[df_concat['Source'] != 'Request B-End'].groupby(['Source', 'Standard Services','Country', 'Commercial Model'])['FCV'].sum()

Series([], Name: FCV, dtype: float64)

In [26]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 22 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   id                                         825 non-null    int64  
 1   quotation_ID                               825 non-null    object 
 2   Source                                     825 non-null    object 
 3   Standard Services                          825 non-null    object 
 4   Site ID                                    825 non-null    object 
 5   Lot                                        825 non-null    object 
 6   City                                       825 non-null    object 
 7   Country                                    825 non-null    object 
 8   Main access speed UpStream (Kbps)          825 non-null    float64
 9   Main access speed DownStream (kbps)        803 non-null    float64
 10  Main access technology    

In [27]:
df_concat.to_csv('concat.csv', index=False)

# Merge:
- bend_summary
- ds_summary
- pe_summary

In [28]:
def rename_columns(df, suffixe_df):
    """Renombrar columnas con el formato deseado, añadiendo un sufijo para saber de qué tabla original proviene.

    Args:
        df (df): DataFrame el cual queremos renombrar las columnas.
        suffixe_df (str): sufijo que añadiremos al nombre de cada columna.
    """
    
    list_columns = df.columns
    list_columns_rename = []

   #Automatically giving the column name: 
    for col in list_columns:
        col = col.replace(' ', '_')
        col_rename = col + '_' + suffixe_df
        list_columns_rename.append(col_rename)
    
    #Automatic dictionary creation for name change:
    dict_names = dict(zip(list_columns, list_columns_rename))

    #Renaming columns:
    df.rename(columns = dict_names, inplace = True)

In [29]:
rename_columns(bend_summary, 'req')
rename_columns(ds_summary, 'ds')
rename_columns(pe_summary, 'pe')

C:\Users\alcorce\AppData\Local\Temp\ipykernel_4496\1007218746.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = dict_names, inplace = True)


In [30]:
#Mergeamos en left (=bend) para que nos aparezcan siempre los ID de quien ha montado el archivo. 
# Aquí podemos ver tb si ha habido desviaciones en los ID.
df_merged_1 = bend_summary.merge(ds_summary, how = 'left', left_on = 'id_req', right_on = 'id_ds')
df_merged_2 = df_merged_1.merge(pe_summary, how = 'left', left_on = 'id_req', right_on = 'id_pe')

# Divisas:

In [31]:
name_to_iso = {
    # América Latina
    "argentine peso": "ARS",
    "bolivian boliviano": "BOB",
    "brazilian real": "BRL",
    "chilean peso": "CLP",
    "colombian peso": "COP",
    "costa rican colon": "CRC",
    "cuban peso": "CUP",
    "dominican peso": "DOP",
    "east caribbean dollar": "XCD",
    "guatemalan quetzal": "GTQ",
    "honduran lempira": "HNL",
    "jamaican dollar": "JMD",
    "mexican peso": "MXN",
    "nicaraguan córdoba": "NIO",
    "panamanian balboa": "PAB",
    "paraguayan guarani": "PYG",
    "peruvian sol": "PEN",
    "trinidad and tobago dollar": "TTD",
    "uruguayan peso": "UYU",
    "venezuelan bolívar": "VES",

    # Otras divisas comunes para completar
    "us dollar": "USD",
    "dollar": "USD",
    "canadian dollar": "CAD",
    "euro": "EUR",
    "british pound": "GBP",
    "japanese yen": "JPY",
    "chinese yuan": "CNY",
    "south korean won": "KRW",
    "indonesian rupee": "IDR"
}


In [32]:
def fcv_currency_or_multicurrency(df, col_currency_req,
                                      col_currency_ds, col_val_ds,
                                      col_currency_pe, col_val_pe,
                                      diccionario):
    """
    Analiza el Full Contract Value de cada quotation en distintas divisas, realizando conversiones a la moneda de referencia (moneda del Request B-End),
    y calcula diferencias porcentuales entre el FCV calculado por el Deal Specialist y el FCV del Pricing Engine. 
    También verifica si las divisas están cubiertas por el diccionario ISO. Habrá que completarlo en el caso de que las divisas no lo estén.

    Args:
        - df (pd.DataFrame): DataFrame con los datos unificados de las tres fuentes: Request B-end, Deal specialist y Pricing Engine.
        - columnas_divisa (list): Lista con los nombres de las columnas de divisas a verificar.
        - col_id (str): número id de cada quotation que une las 3 fuentes (B-end, DS, PE).
        - col_currency_req (str): Columna con la divisa de referencia para la quotation (Request B-End).
        - col_currency_ds (str): Columna de divisa del Deal Specialist.
        - col_val_ds (str): Columna con el PCV del DS.
        - col_currency_pe (str): Columna de divisa del Pricing Engine.
        - col_val_pe (str): Columna con el PCV del PE.
        - diccionario (dict): Diccionario con las equivalencias 'nombre de divisa' → 'código ISO' (ej. "euro" → "EUR").

    Returns:
        - pd.DataFrame: DataFrame con valores convertidos, tasas de cambio y delta %.
    """

    # PASO 1: Verificar si las divisas existen en el diccionario
    columnas_divisa = [col_currency_req, col_currency_ds, col_currency_pe]
    todas_divisas = set()
    for col in columnas_divisa:
        if col in df.columns:
            divisas = df[col].dropna().unique()
            todas_divisas.update(divisas)

    print("🔍 Revisión de nombres de divisa encontrados:\n")
    for divisa in sorted(todas_divisas):
        clave = divisa.strip().lower()
        if clave in diccionario:
            print(f" ✅'{clave}' → {diccionario[clave]}")
        else:
            print(f"❌'{clave}' → ❗ FALTA en el diccionario de divisas")


    # PASO 2: Configurar caché para no repetir llamadas a la API
    cache_tipo_cambio = {}

    def name_to_iso(nombre_divisa):
        """Convierte un nombre de divisa en texto (por ejemplo, 'euro', 'dollar') a su correspondiente código ISO de tres letras 
        (por ejemplo, 'EUR', 'USD'), utilizando un diccionario de mapeo previamente definido. Funcionamiento:
            1. Convierte el nombre a minúsculas.
            2. Elimina espacios en blanco al inicio y final.
            3. Busca el valor en el diccionario `diccionario`.
            4. Devuelve el código ISO si lo encuentra, o None si no existe.

        Args:
            nombre_divisa (str): nombre de la divisa (no estándarizado). Ejemplos: "Euro", " dollar ", "CANADIAN DOLLAR"

        Returns:
            str or None: Código ISO. Devuelve None si la divisa no está en el diccionario.
        """
        return diccionario.get(str(nombre_divisa).strip().lower(), None)

    def obtener_tasa_cambio(origen, destino):
        """Dado un par de monedas (ej.: "USD" a "EUR") esta función:
            (1) Consulta la API FXRatesAPI para saber el tipo de cambio actual.
            (2) Guarda los resultados en memoria temporal (cache_tipo_cambio) para no repetir la llamada en caso de que se repita el par de currencies para los que necesitamos conversión.
            (3) Devuelve el exchange rate para que luego se use en la conversión de "presupuestos".

        Args:
            origen (str): código de divisa origen (ej.: "USD")
            destino (str): código de divisa destino (ej.: "EUR")

        Returns:
            float: tipo de cambio obtenido de la API. Lo aplicamos con un ejemplo:
                    Ej. obtener_tasa_cambio("USD", "EUR")  output--> 0.8672701405
        """

        #CASO 1: no se necesita conversión
        if origen == destino:
            return 1
        
        #CASO 2: se necesita conversión y tenemos almacenado en el caché el tipo de cambio
        par = (origen, destino) #Guardamos el par de divisas como una tupla (ej.: ("USD", "EUR")).
        if par in cache_tipo_cambio: #Verificamos si ya tenemos esa conversión guardada (cache)
            return cache_tipo_cambio[par]  #Si ya lo hemos consultado antes, devolvemos el exchange rate desde el diccionario cache_tipo_cambio. Esto ahorra muchas llamadas a la API.
        
        #CASO 3: Llamamos a la API si no encuentra el tipo de cambio en el cache
        url = f"https://api.fxratesapi.com/convert?from={origen}&to={destino}&format=json"
        
        try:
            r = requests.get(url, timeout=10) ##Esperamos un máximo de 10 segundos evitando que el script se quede colgado indefinidamente si el servidor no responde.
            if r.status_code == 200:

                data = r.json() #convertimos la respuesta exitosa a un dict de Python (JSON)
                tasa = data.get("info", {}).get("rate", None) #Extraemos del JSON el valor del tipo de cambio (que está en "info": {....., "rate": 0.923421},) Si no existe, nos da None.
                cache_tipo_cambio[par] = tasa #Guardamos la tasa en el cache para futuros usos
                
                return tasa
        
        #CASO 4: Si hay algún error (red, timeout, etc.), lo imprime y devuelve None.
        except Exception as e:
            print(f"Error al obtener tipo de cambio {origen} --> {destino}: {e}")
        
        return None


    def convertir(valor, origen, destino):
        """convertimos un valor monetario de una divisa a otra, usando una tasa de cambio obtenida previamente de la API (y cacheada para eficiencia).

        Args:
            valor (float): importe que se quiere convertir (ej. 100)
            origen (str): Código ISO de la moneda del valor original (ej. "USD")
            destino (str): ódigo ISO de la moneda a la que se quiere convertir (ej. "EUR")

        Returns:
            tuple: Devuelve una tupla con (el valor convertido , la tasa de cambio utilizada). 
                    Ej. convertir(100, 'USD', 'EUR') 
                    Supongamos que obtener_tasa_cambio (PASO ANTERIOR) devuelve 0.92 --> resultado convertir(100, 'USD', 'EUR') --> (92.0, 0.92)
        """
        #CASO 1: si alguno de los tres parámetros es None, no se podrá hacer la conversión
        if valor is None or origen is None or destino is None:
            return None, None
        
        #CASO 2: cuando sí podemos obteber la conversión:
        tasa = obtener_tasa_cambio(origen, destino)
        valor_convertido = valor * tasa if tasa else None #Si la tasa es válida (no None), se multiplica por el valor original. Si no hay tasa, el resultado es None.
        return valor_convertido, tasa #Devuelve una tupla: el valor convertido y la tasa de cambio utilizada


    def identificar_cambio_divisa(exchange_rate):
        """Creamos una nueva columna identificando si la divisa utilizada por el Deal Specialist y por el Pricing Engine es la misma que la requerida en el Request B-End.

        Args:
            exchange_rate (float or None): Tipo de cambio aplicado entre las dos monedas

        Returns:
            str: Tenemos 3 posibilidades:
                (1) "no currency available" --> Si el tipo de cambio es NaN (no disponible)
                (2) "same currency as B-end" --> Las divisas son iguales (porque el tipo de cambio = 1).
                (3) "different currency as B-end" --> Las divisas son diferentes, porque el tipo de cambio es distinto de 1.
        """
        if pd.isna(exchange_rate):
            return "no currency available"
        elif exchange_rate == 1:
            return "same currency as B-end"
        else:
            return "different currency as B-end"



    # PASO 3: Aplicar lógica de conversión
    df = df.copy()

    #1) llamamos a la función name_to_iso para convertir la divisa a Código ISO para poder hacer la llamada a la API
    df['currency_ISO_req'] = df[col_currency_req].apply(name_to_iso)
    df['currency_ISO_ds'] = df[col_currency_ds].apply(name_to_iso)
    df['currency_ISO_pe'] = df[col_currency_pe].apply(name_to_iso)

    # llamamos a la función convertir(FCV, divisa_origen, divisa_destino) para convertir los FCV del DS y PE en la divisa de referencia (=la del Request B-End). Aquí estamos haciendo llamamiento a la API
    #2) Guardamos el FCV en la divisa de referencia:
    df['FCV_ds_conv'] = df.apply(lambda row: convertir(row[col_val_ds], row['currency_ISO_ds'], row['currency_ISO_req'])[0], axis=1)
    df['FCV_pe_conv'] = df.apply(lambda row: convertir(row[col_val_pe], row['currency_ISO_pe'], row['currency_ISO_req'])[0], axis=1)

    #3) Guardamos el tipo de cambio aplicado por si necesitamos recurrir a él:
    df['FCV_ds_exchange_rate'] = df.apply(lambda row: convertir(row[col_val_ds], row['currency_ISO_ds'], row['currency_ISO_req'])[1], axis=1)
    df['FCV_pe_exchange_rate'] = df.apply(lambda row: convertir(row[col_val_pe], row['currency_ISO_pe'], row['currency_ISO_req'])[1], axis=1)

    #4) Cálculo de delta:(Valor actual - Valor de referencia) / Valor de referencia * 100 o bien Valor actual / Valor de referencia - 1.
    #En este caso, nuestro benchmark es el Deal Specialist
    df['delta PE vs DS'] = ((df['FCV_pe_conv'] / df['FCV_ds_conv']) - 1) * 100

    #5) Reemplazamos los infinitos a 0%. Estos infinitos pueden existir porque el FCV del benchmark (DS) puede ser == 0, entonces al calcular la delta de FCV sale inf:
    df['delta PE vs DS'] = df['delta PE vs DS'].replace([np.inf, -np.inf], 0)

    #6) identificamos si la divisa utilizada por el Deal Specialist y por el Pricing Engine es la misma que la requerida en el Request B-End
    df['same_currency_as_B-End_ds'] = df['FCV_ds_exchange_rate'].apply(identificar_cambio_divisa)
    df['same_currency_as_B-End_pe'] = df['FCV_pe_exchange_rate'].apply(identificar_cambio_divisa)

    return df

In [33]:
df_merged_final = fcv_currency_or_multicurrency(
    df_merged_2,
    col_currency_req = 'Main_Access_Currency_req',
    col_currency_ds = 'Main_Access_Currency_ds', col_val_ds = 'FCV_ds',
    col_currency_pe = 'Main_Access_Currency_pe', col_val_pe = 'FCV_pe',
    diccionario = name_to_iso
)

🔍 Revisión de nombres de divisa encontrados:

 ✅'dollar' → USD


# Comprobación de delta:


In [34]:
#porcentaje de nulos sin completar:
df_merged_final[['FCV_pe','FCV_ds', 'delta PE vs DS']].isnull().sum()/df_merged_final.shape[0]*100

FCV_pe            0.0
FCV_ds            0.0
delta PE vs DS    0.0
dtype: float64

In [35]:
def same_commercial_model_quoted (value):
    """función que identifica si ha habido cambios entre el servicio precidado por el PE y los DS en la columna "Comments_VPN_Site_Info\n/Commercial_Model" cuyos dos únicos valores sólo pueden ser B4B y DIA. 
    Esta función compara dos columnas: 
    (1) Commercial_Model_req (antigua columna Comments_VPN_Site_Info\n/Commercial_Model) : columna descriptiva del Commercial Model solicitado en el B-End
    (2) Commercial_Model_ds (antigiua columna Comments_VPN_Site_Info\n/Commercial_Model): columna descriptiva del Commercial Model preciado por los Deal Specialists
    (3) Commercial_Model_pe (antigua columna Comments_VPN_Site_Info\n/Commercial_Model): columna descriptiva del Commercial Model preciado por el Pricing engine

    Args:
        value (string): celda dentro de cada columna, es decir, tipo de servicio preciado por cada agente.

    Returns:
        string: devuelve 
        (1)"No changes": si no ha habido cambio del commercial model
        (2) "PE changed": si quien ha cambiado el commercial model quoted ha sido el Pricing Engine
        (3) "DS changed": si quien ha cambiado el commercial model quoted ha sido el Deal Specialist.
    """
    #Extraemos el tipo de Commercial Model de cada una de las 3 columnas para esta fila (value)
    b_end = value['Commercial_Model_req']
    pe = value['Commercial_Model_pe']
    ds = value['Commercial_Model_ds']

    if b_end == "DIA" or b_end == "B4B":
        #Creamos una lista vacía donde vamos a ir guardando quién cambió el servicio. OJO el PE puede ser distinto al B-end pero también puede serlo el DS.
        cambios = []

        #Si el commercial model del PE es distinto del del B-end --> agregamos mensaje a la lista
        if pe != b_end: 
            cambios.append("PE changed") 

        #Si el commercial model del DS es distinto del del B-end --> agregamos mensaje a la lista
        if ds != b_end: 
            cambios.append("DS changed") 

        #CASO 1: Si la lista sigue vacía --> significa que PE y DS coinciden con B-end --> no hubo cambios en Commercial model
        if not cambios:
            return 'No changes'
        
        #CASO 2: Si hay uno o más cambios --> los unimos con "&" 
        return " & ".join(cambios)

    else: #CASO 3: que no estemos ante un commercail model (B4B o DIA, si no que estemos ante un MPLS, por ejemplo)
        return "Other service quoted (i.e. MPLS)"

In [36]:
df_merged_final['Commercial_model_changes'] = df_merged_final.apply(same_commercial_model_quoted, axis=1)

In [37]:
df_merged_final[['Commercial_Model_req', 'Commercial_Model_ds', 'Commercial_Model_pe', 'Commercial_model_changes']].head()

,Commercial_Model_req,Commercial_Model_ds,Commercial_Model_pe,Commercial_model_changes
0,B4B,B4B,DIA,PE changed
1,DIA,DIA,DIA,No changes
2,B4B,B4B,DIA,PE changed
3,DIA,DIA,DIA,No changes
4,B4B,B4B,DIA,PE changed


In [38]:
df_precios = df_merged_final[['id_req', 'Site_ID_req', 'City_req', 'Country_req', 'Commercial_Model_req', 'currency_ISO_req', 'Main_Access_Provider_(last_mile_Provider)_ds', 'FCV_ds_conv', 'same_currency_as_B-End_ds','Commercial_Model_ds','Commercial_Model_pe', 'Main_Access_Provider_(last_mile_Provider)_pe', 'main_access_mrc_amt_quoted_by_pe', 'FCV_pe_conv', 'same_currency_as_B-End_pe', 'Commercial_model_changes', 'delta PE vs DS']]

In [39]:
df_merged_final.to_csv('merged_viz.csv', index = False) #lo guardamos para visualizaciones

In [40]:
#Preparación de datos para el Power BI, cambio los puntos de todas las columnas con floats por las comas, para que el Power BI lo entienda:
def preparacion_floats_powerbi (df): 
    """Función para la preparación de datos para el Power BI: cambiando los puntos de todas las columnas con floats por las comas, para que el Power BI lo entienda:

    Args:
        df (DataFrame): DataFrame que queremos hacer someter al cambio.
    """
    for col in df.columns:
        if df[col].dtype == np.float64:
            df[col] = df[col].apply(lambda x: str(x) if isinstance(x, (int, float)) else x)
            df[col] = df[col].str.replace(".", ",")

In [41]:
preparacion_floats_powerbi(df_precios)
preparacion_floats_powerbi(df_merged_final)

C:\Users\alcorce\AppData\Local\Temp\ipykernel_4496\1879978361.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: str(x) if isinstance(x, (int, float)) else x)
C:\Users\alcorce\AppData\Local\Temp\ipykernel_4496\1879978361.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace(".", ",")


In [42]:
df_merged_final[['Country_req', 'Commercial_Model_req','Commercial_Model_ds', 'Commercial_Model_pe', 'Commercial_model_changes']][df_merged_final['Country_req'] == 'Argentina']

,Country_req,Commercial_Model_req,Commercial_Model_ds,Commercial_Model_pe,Commercial_model_changes
0,Argentina,B4B,B4B,DIA,PE changed
1,Argentina,DIA,DIA,DIA,No changes
2,Argentina,B4B,B4B,DIA,PE changed
3,Argentina,DIA,DIA,DIA,No changes
4,Argentina,B4B,B4B,DIA,PE changed
5,Argentina,DIA,DIA,DIA,No changes


In [43]:
df_merged_final.to_csv('merged.csv', index = False)
df_precios.to_csv('precios.csv', index = False)